Our goal is to parse a wiki dump from https://cinemorgue.fandom.com/wiki/Special:Statistics to figure out which actor/actress has died the most in movies because I don't believe a single click-bait article I've found online. Trust but verify ya know.

If you wish to grab a more recent update, download the "Current Pages" to get an XML file.
Do not download "Current Page and History" unless you're a bit of a parsing masochist.

Fandom (the site Cinemorgue is hosted on) follows the mediawiki export format of /export-0.11/.
Using the xml tree is way easier than relying on any sort of wikiparser library out there.

In [1]:
import xml.etree.ElementTree as ET
import pandas as pd

NS = 'http://www.mediawiki.org/xml/export-0.11/'

def parse_wikimedia_xml(filepath):
    tree = ET.parse(filepath)
    root = tree.getroot()
    data = []
    for page in root.findall('{%s}page' % NS):
        ns = page.find('{%s}ns' % NS).text
        if ns != "0":
            continue
        title = page.find('{%s}title' % NS).text
        revision = page.find('{%s}revision' % NS)
        text = revision.find('{%s}text' % NS).text
        data.append({'title': title, 'text': text})
    df = pd.DataFrame(data)
    return df

df = parse_wikimedia_xml('input/cinemorgue_8.3.23.xml')

df.head(3)

,title,text
0,Cinemorgue Wiki,<mainpage-leftcolumn-start />\n{{Mainpage welc...
1,Main Page,#REDIRECT [[Cinemorgue Wiki]]
2,Marilyn Monroe,[[File:Marilynmonroe.jpg|frame|Marilyn Monroe ...


We will only be looking at Film Deaths; TV Deaths seem to include a lot of voice actors from animated shows which feels like cheating and ruins the spirt of finding out which actor has died the most. 

The structure of each wikimedia page, while not perfect, is relatively consistent.

Each Actor/Actress page follows the structure below:

Overview 
Film Deaths 
Television Deaths/TV Deaths 
Video Game Deaths 
Music Video Deaths 
Notable Connections 

Not every page has the subsequent sections, so just to be thorough, we check for and delete every other section.

In [2]:
# Delete everything before Film Deaths.
df['text'] = df['text'].str.split("Film Deaths", n=1, expand=True)[1]

In [3]:
#Delete everything below TV Deaths.
df['text'] = df['text'].str.split("Television Deaths", n=1, expand=True)[0]

In [4]:
df['text'] = df['text'].str.split("TV Deaths", n=1, expand=True)[0]

In [5]:
df['text'] = df['text'].str.split("TV Series Deaths", n=1, expand=True)[0]

In [6]:
df['text'] = df['text'].str.split("Video Game Deaths", n=1, expand=True)[0]

In [7]:
df['text'] = df['text'].str.split("Music Video Deaths", n=1, expand=True)[0]

In [8]:
df['text'] = df['text'].str.split("Notable Connections", n=1, expand=True)[0]

In [9]:
df['text'] = df['text'].str.split("Noteworthy Connections", n=1, expand=True)[0]

In [10]:
df['text'] = df['text'].str.split("Gallery", n=1, expand=True)[0]

In [11]:
#Drop all recently nulled rows. Ready for splitting.
df = df.dropna(subset=['text'])

Every movie death is (generally) annotated by a line break and an asterisk.
For each instance of this, we'll the entry out into a new row.

In [12]:
# New DataFrame to store the split rows
new_rows = {'title': [], 'text': []}

# Iterate through the original DataFrame
for idx, row in df.iterrows():
    title = row['title']
    text_parts = row['text'].split('\n*')
    
    # Append the new rows to the new DataFrame
    # Skip the first element, usually contains gibberish before first line.
    for part in text_parts[1:]:
        new_rows['title'].append(title)
        new_rows['text'].append(part)

# Create the new DataFrame
new_df = pd.DataFrame(new_rows)

# Print the result
new_df.head(3)

,title,text
0,Joseph Cotten,'''[[Shadow of a Doubt (1943)|''Shadow of a Do...
1,Joseph Cotten,'''''[[Niagara (1953)]]''''' [''George Loomis'...
2,Joseph Cotten,'''''The Last Sunset'' (1961)''' [''John Breck...


Now we need to find the year the film was released. 
This is to help differentiate common/repeated movie titles that have been made over the year.

We are looking for the first instance of 4 digits between parentheis.

In [13]:
#Creating year column.
import re

def extract_year(text):
    match = re.search(r'\((\d{4})\)', text)
    if match:
        return match.group(1)
    else:
        return None

# Apply the function to create the "year" column
new_df['year'] = new_df['text'].apply(extract_year)

new_df.head(3)

,title,text,year
0,Joseph Cotten,'''[[Shadow of a Doubt (1943)|''Shadow of a Do...,1943
1,Joseph Cotten,'''''[[Niagara (1953)]]''''' [''George Loomis'...,1953
2,Joseph Cotten,'''''The Last Sunset'' (1961)''' [''John Breck...,1961


Now begins the slow stripping away of unncessary info following each movie title.

There are a lot of weird cases, and even weirder non alphanumeric characters. 
So we're going to strip away characters slowly but surely to create uniform pattern we can parse.

In [14]:
#Remove all pairs of apostraphes or quotation marks.
new_df['text'] = new_df['text'].str.replace(r"[''\"\=]", "", regex=True)

In [15]:
# Some titles have stray html formatting tags in them. 
new_df['text'] = new_df['text'].str.replace(r'\s*<.*?>\s*', '', regex=True)

In [16]:
#Some titles will just have the link hardcoded in the title which is pretty impressive.
new_df['text'] = new_df['text'].str.replace(r'https://\S+\s*', '', regex=True)

In [17]:
#Some titles might even hardcode the unsecured link instead.
new_df['text'] = new_df['text'].str.replace(r'http://\S+\s*', '', regex=True)

Titles come in two forms: Links and Non-Links.

Links are formatted in a way that have the title listed twice between brackets.
(e.g. [The Shining (1980) | The Shining (1980)])

Non-Links bow to no god, and do whatever they want to do. 
Our best hope is to just capture everything leading up to the first instance of a year between parenthesis. (e.g. ________ (1980) )

In [18]:
# If a string starts with a link [], grab the contained string.
# If a string is not a link, grab all textup until the first date ().
def extract_text(row):
    if row.startswith("["):
        # Remove parenthesis and their contents from inside the square brackets
        cleaned_text = re.sub(r'\([^()]*\)', '', row)
        match = re.search(r'\[(.*?)\]', cleaned_text)
        if match:
            return match.group(1)
    else:
        match = re.search(r'^([^()]*)', row)
        if match:
            return match.group(1).strip()
    return ''  # Return an empty string if no match is found

new_df['text'] = new_df['text'].apply(extract_text)

In [19]:
# For links, delete everything after the first instance of a |.
new_df['text'] = new_df['text'].str.split("|", n=1, expand=True)[0]

In [20]:
# Remove all remaining [[]].
new_df['text'] = new_df['text'].str.replace(r'\[|\]', '', regex=True)

In [21]:
#Categories sneak their way into some titles, so remove these as well.
new_df['text'] = new_df['text'].str.split("{", n=1, expand=True)[0]

In [22]:
# Remove all white space at end of string.
new_df['text'] = new_df['text'].str.strip()

In [23]:
# Remove all blank rows.
new_df = new_df[new_df['year'] != '']

In [24]:
#Remove all null rows that don't contain a year.
new_df = new_df.dropna(subset=['year'])

In [25]:
#Final export.
new_df.to_csv('output/Cinemorgue.csv', index=False)

In [26]:
new_df.head(3)

,title,text,year
0,Joseph Cotten,Shadow of a Doubt,1943
1,Joseph Cotten,Niagara,1953
2,Joseph Cotten,The Last Sunset,1961
